In [1]:
import ee
import geemap
import math
ee.Initialize()

<h2> Imports </h2>

<h4> Satellites </h4>

In [47]:
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
admin = ee.FeatureCollection("FAO/GAUL/2015/level0")
active_mines = ee.FeatureCollection("users/rishiAgarwal/Congo_Active_Mines")

<h4> Regions </h4>

In [55]:
rayroi1 = ee.Geometry.Polygon(
        [[[29.554129272985683, 3.1591674847348235],
          [29.554129272985683, 3.092319151883147],
          [29.625197083044277, 3.092319151883147],
          [29.625197083044277, 3.1591674847348235]]])

rayroi2 = ee.Geometry.Polygon(
        [[[30.246670960050185, 1.7911944738716732],
          [30.246670960050185, 1.7103797163160706],
          [30.356362579923232, 1.7103797163160706],
          [30.356362579923232, 1.7911944738716732]]])

rayall = ee.Geometry.Polygon(
        [[[22.92587933089792, 5.5911393992628495],
          [22.92587933089792, -13.776973382582892],
          [31.45126995589792, -13.776973382582892],
          [31.45126995589792, 5.5911393992628495]]])

emily_area = ee.Geometry.Polygon(
        [[[25.599767416235804, -10.585427828449394],
          [25.599767416235804, -10.82359555219894],
          [25.977297323462366, -10.82359555219894],
          [25.977297323462366, -10.585427828449394]]])

focus = ee.Geometry.Polygon(
        [[[27.350233348102517, -7.518171474050515],
          [27.350233348102517, -7.57841301205225],
          [27.436407359332986, -7.57841301205225],
          [27.436407359332986, -7.518171474050515]]], None, False)

<h4> Visuzalitaions </h4>

In [4]:
rgb_vis_ls5 = {
    'min': 8211.7,
    'max': 11077.3,
    'bands': ['SR_B3', 'SR_B2', 'SR_B1']
}

rgb_vis_s2 = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

<h4> Classification Points </h4>

In [5]:
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.566649352977876, 3.131936594576548]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.594069479589553, 3.143136886698536]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.578229613742543, 3.109209943820683]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.581158585986806, 3.1067030888196494]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.55884031463487, 3.11517956941471]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.581819366025293, 3.132088091449441]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.595938514279688, 3.131445322675145]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.59962923388418, 3.1347448648629315]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.565039466428125, 3.104877202480009]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.607741346488673, 3.1193612914096605]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

In [6]:
bare_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.588761955782978, 3.110552597531067]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.590113789126484, 3.1101669284095776]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.570400011069342, 3.114240974803364]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.56574369621949, 3.114048140935224]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.566300979571633, 3.1296952794920045]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.61279975505381, 3.134623171579494]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.620696178393654, 3.1514206812868597]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.60730659099131, 3.15656272195781]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.609366527514748, 3.1041127202608414]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.561301341967873, 3.0982847807562592]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

<h3> Training the Classifier </h3>

In [7]:
training = bare.merge(vegetation)

composite1985 = ls5 \
        .filter(ee.Filter.bounds(rayroi1)) \
        .filter(ee.Filter.date('1985-01-01', '1986-01-01')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .select('SR_B.*') \
        .median() \
        .clip(rayroi1)

# Overlay the point on the image to get training data.
training = composite1985.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# Train a classifier.
classifier_ls = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite1985.bandNames()
})

# Classify the image.
classified = composite1985.classify(classifier_ls)

In [8]:
training = bare_s.merge(vegetation_s)

composite2021 = s2 \
        .filter(ee.Filter.bounds(rayroi1)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(rayroi1) \

# Overlay the point on the image to get training data.
training = composite2021.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# Train a classifier.
classifier_s2 = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite2021.bandNames()
})

# Classify the image.
classified = composite2021.classify(classifier_s2)

# Map.addLayer(classified,
#   {'min': 0, 'max': 1, 'palette': ['brown', 'green']}, '2021 S2 Classified')

# Map

<h2> Calculation Methods</h2>

<h4> Calculate Percent Veg Change in a Region </h4>

In [9]:
def calculate_percentage_change(feature):
    g = feature.geometry()

    composite_ls = ls5 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .select('SR_B.*') \
        .median() \
        .clip(g)
    
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(g) \
    
    left_classified = composite_ls.classify(classifier_ls)
    right_classified = composite_s2.classify(classifier_s2)
    
    # Reclassify from 0-1 to 1-2
    left_classes = left_classified.remap([0, 1], [1, 2])
    right_classes = right_classified.remap([0, 1], [1, 2])

    initial_vegetation = left_classes.eq(2)
    bare = right_classes.eq(1)
    vegetation_to_bare = initial_vegetation.And(bare)
    
    area_image = vegetation_to_bare.multiply(ee.Image.pixelArea())

    area = area_image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 100,
      'maxPixels': 1e10
    })

    area_vegetation_to_bare = ee.Number(area.get('remapped')).divide(1e6)

    area_initial_vegetation = left_classified.eq(1).multiply(ee.Image.pixelArea())

    area2 = area_initial_vegetation.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': g,
      'scale': 100,
      'maxPixels': 1e10
    })

    area_initial_vegetation = ee.Number(area2.get('classification')).divide(1e6)
    
    percent_loss = area_vegetation_to_bare.divide(area_initial_vegetation).multiply(100)
    
    return feature.set('percent loss',  percent_loss)

<h4> Calculate NIR/G Value in a Region </h4>

In [28]:
def calculate_nirg_value(feature):
    g = feature.geometry()
    composite_s2 = s2 \
        .filter(ee.Filter.bounds(g)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*') \
        .median() \
        .clip(g) 
    nirg = composite_s2.normalizedDifference(['B8', 'B3']).rename('nirg')
    stats = nirg.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': g,
        'scale': 100,
        'maxPixels': 1e10
    })
    nirg_val = stats.get('nirg')
    
    return feature.set('nirg', nirg_val)

<h3> Divide Given Geometry to Specified Square Size (in Km) </h3>

In [29]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:return: list including all squares
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]], None, False)
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

<h2> Routine </h2>

<h5> Filtering the sqaures by percent of veg loss </h5>

In [30]:
def filter_by_vegetation_loss(squares, threshold):
    with_percent_change = squares.map(calculate_percentage_change)
    passed = with_percent_change.filter(ee.Filter.gt('percent loss', threshold))
    return passed

In [31]:
def create_median_composite(feature):
    filtered = s2 \
        .filter(ee.Filter.bounds(feature.geometry())) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .select('B.*')
    
    composite = filtered.median().clip(feature.geometry())
    return composite

In [40]:
def filter_by_nirg(squares, threshold):
    with_nirg = squares.map(calculate_nirg_value)
    passed = with_nirg.filter(ee.Filter.lte('nirg', threshold))
    return passed

<h3> Pasting onto the Map </h3>

<h5> Rishi Region </h5>

In [50]:
Map = geemap.Map()
Map.centerObject(focus, 13)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(focus, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_nirg = filter_by_nirg(passed_vegetation_loss, 0.45)
# read = passed_vegetation_loss.getInfo()['features']
# for item in read:
#     print(item['properties'])

composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(passed_nirg, {'color': 'blue'}, 'Passed Nirg greater than 0', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[-7.54829224824849, 27.393320353717847], controls=(WidgetControl(options=['position', 'transparent_…

<h5> Ray Region </h5>

In [51]:
Map = geemap.Map()
Map.centerObject(rayroi1, 13)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(rayroi1, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_nirg = filter_by_nirg(passed_vegetation_loss, 0.45)
# read = passed_vegetation_loss.getInfo()['features']
# for item in read:
#     print(item['properties'])

composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(passed_nirg, {'color': 'blue'}, 'Passed Nirg greater than 0', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[3.1257435633122834, 29.589663178011996], controls=(WidgetControl(options=['position', 'transparent…

In [53]:
Map = geemap.Map()
Map.centerObject(rayroi2, 13)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(rayroi2, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_nirg = filter_by_nirg(passed_vegetation_loss, 0.45)
# read = passed_vegetation_loss.getInfo()['features']
# for item in read:
#     print(item['properties'])

composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(passed_nirg, {'color': 'blue'}, 'Passed Nirg greater than 0', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[1.7507876063752705, 30.30151676998732], controls=(WidgetControl(options=['position', 'transparent_…

<h5> Emily Region </h5>

In [57]:
Map = geemap.Map()
Map.centerObject(emily_area, 12)

drc = admin.filter(ee.Filter.eq('ADM0_NAME', 'Democratic Republic of the Congo'))

segments = ee.FeatureCollection(create_segments(emily_area, 0.5)).filter(ee.Filter.bounds(drc))

passed_vegetation_loss = filter_by_vegetation_loss(segments, 20)
passed_nirg = filter_by_nirg(passed_vegetation_loss, 0.45)
# read = passed_vegetation_loss.getInfo()['features']
# for item in read:
#     print(item['properties'])

composites = ee.ImageCollection(segments.map(create_median_composite))

Map.addLayer(composites, rgb_vis_s2, 'Median Composites 2021')
Map.addLayer(passed_vegetation_loss, {'color': 'red'}, 'Passed 20% Vegetation Loss', opacity=0.7)
Map.addLayer(passed_nirg, {'color': 'blue'}, 'Passed Nirg greater than 0', opacity=0.7)
Map.addLayer(active_mines, {'color': 'green'}, 'mapbox active mines')

Map

Map(center=[-10.704552846449875, 25.788532369849722], controls=(WidgetControl(options=['position', 'transparen…